In [3]:
# -*- coding: utf-8 -*- #

import os
os.environ['CUDA_VISIBLE_DEVICES'] = "3"
import tensorflow as tf
import numpy as np
import time
import random

from babi_loader import QAdata
from qa_lstm_model import Model


def initialize_session():
    config = tf.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = 0.4
    return tf.Session(config=config)


##################################################
BATCH_SIZE = 20         # 배치 사이즈
emb_dim = 64            # 단어 embedding dimension
hidden_dim = 64         # RNN hidden dim

max_t_len = 80
max_q_len = 20

learning_rate = 0.0005    # Learning rate
use_clip = True         # Gradient clipping 쓸지 여부
##################################################

data = QAdata(path="./dataset/babi", max_t_len=max_t_len, max_q_len=max_q_len)
model = Model(emb_dim=emb_dim, hidden_dim=hidden_dim, vocab_size=data.vocab_size)

sess = initialize_session()
sess.run(tf.global_variables_initializer())

def sample_test():
    T, Q, A, t_len, q_len = data.get_test(1)
    feed_dict = {model.T: T, model.Q: Q, model.t_len: t_len, model.q_len: q_len}
    output = sess.run(model.output, feed_dict=feed_dict)

    t_line = data.ids2sent(T[0])
    q_line = data.ids2sent(Q[0])
    true_A = data.idx2w[A[0]]
    out_A = data.idx2w[output[0]]

    print(" - Given text: {}".format(t_line))
    print("  - Question: {}".format(q_line))
    print("   - Answer: {} / {} (ground-truth)\n".format(out_A, true_A))


def test_model():
    num_it = 10
    test_loss, test_cnt, test_right = 0, 0, .0

    for _ in range(num_it):
        T, Q, A, t_len, q_len = data.get_test(BATCH_SIZE)
        feed_dict = {model.T: T, model.Q: Q, model.A: A,
                     model.t_len: t_len, model.q_len: q_len}
        loss, output = sess.run([model.loss, model.output], feed_dict=feed_dict)
        test_loss += loss
        test_cnt += 1
        for i, o in enumerate(output):
            if o == A[i]:
                test_right += 1
    print(" * test loss: {:.3f} | acc: {:.3f}\n".format(test_loss / test_cnt, test_right / test_cnt / BATCH_SIZE))



print(" * Data size: ", data.total_size)
print(" * Vocab size: ", data.vocab_size)

avg_loss, it_cnt = 0, 0
it_log, it_test, it_save, it_sample = 10, 50, 1000, 50
start_time = time.time()

for it in range(0, 10000):
    T, Q, A, t_len, q_len = data.get_train(BATCH_SIZE)
    feed_dict = {model.T: T, model.Q: Q, model.A: A,
                 model.t_len: t_len, model.q_len: q_len}
    loss, _ = sess.run([model.loss, model.update], feed_dict=feed_dict)

    avg_loss += loss
    it_cnt += 1

    if it % it_log == 0:
        print(" it: {:4d} | loss: {:.3f} - {:.2f}s".format(it, avg_loss / it_cnt, time.time() - start_time))
        avg_loss, it_cnt = 0, 0

    if it % it_test == 0 and it > 0:
        test_model()
    if it % it_save == 0 and it > 0:
        model.save(sess)
    if it % it_sample == 0 and it > 0:
        sample_test()


 * Data size:  14134
 * Vocab size:  36
 it:    0 | loss: 3.577 - 0.74s
 it:   10 | loss: 3.466 - 1.66s
 it:   20 | loss: 2.636 - 2.46s
 it:   30 | loss: 1.891 - 3.26s
 it:   40 | loss: 1.845 - 3.95s
 it:   50 | loss: 1.832 - 4.67s
 * test loss: 1.804 | acc: 0.205

 - Given text:  Mary journeyed to the garden . Daniel moved to the garden . John went back to the garden . Daniel moved to the office . John went to the office . John went back to the kitchen . John moved to the bedroom . John moved to the garden . <eos> 
  - Question:  Where is John ? <eos> 
   - Answer: bedroom / garden (ground-truth)

 it:   60 | loss: 1.830 - 5.94s
 it:   70 | loss: 1.828 - 6.65s
 it:   80 | loss: 1.823 - 7.41s
 it:   90 | loss: 1.809 - 8.19s
 it:  100 | loss: 1.812 - 8.94s
 * test loss: 1.798 | acc: 0.140

 - Given text:  Sandra went back to the bedroom . John travelled to the office . Sandra journeyed to the hallway . Daniel went to the bedroom . Sandra went to the bedroom . Mary travelled to the offic

 * test loss: 1.151 | acc: 0.540

 - Given text:  Sandra travelled to the office . Daniel journeyed to the hallway . Sandra went to the hallway . Mary went to the garden . Mary travelled to the kitchen . John travelled to the garden . Sandra journeyed to the bathroom . Mary travelled to the bathroom . John moved to the bathroom . Mary went to the office . <eos> 
  - Question:  Where is John ? <eos> 
   - Answer: bathroom / bathroom (ground-truth)

 it:  860 | loss: 1.184 - 73.65s
 it:  870 | loss: 1.244 - 74.47s
 it:  880 | loss: 1.295 - 75.27s
 it:  890 | loss: 1.169 - 76.02s
 it:  900 | loss: 1.182 - 76.91s
 * test loss: 1.258 | acc: 0.455

 - Given text:  John went to the garden . John journeyed to the bathroom . Daniel went to the hallway . Daniel journeyed to the garden . <eos> 
  - Question:  Where is John ? <eos> 
   - Answer: garden / bathroom (ground-truth)

 it:  910 | loss: 1.269 - 78.18s
 it:  920 | loss: 1.160 - 78.98s
 it:  930 | loss: 1.120 - 79.78s
 it:  940 | loss: 1.1

 it: 1660 | loss: 1.177 - 149.62s
 it: 1670 | loss: 1.254 - 150.53s
 it: 1680 | loss: 1.104 - 151.44s
 it: 1690 | loss: 1.157 - 152.37s
 it: 1700 | loss: 1.156 - 153.32s
 * test loss: 1.054 | acc: 0.540

 - Given text:  John went back to the garden . Sandra moved to the office . John moved to the hallway . John went back to the office . <eos> 
  - Question:  Where is Sandra ? <eos> 
   - Answer: office / office (ground-truth)

 it: 1710 | loss: 1.082 - 154.62s
 it: 1720 | loss: 1.066 - 155.52s
 it: 1730 | loss: 1.048 - 156.43s
 it: 1740 | loss: 1.087 - 157.39s
 it: 1750 | loss: 1.162 - 158.28s
 * test loss: 1.072 | acc: 0.585

 - Given text:  John went back to the kitchen . Daniel journeyed to the hallway . Sandra went to the bathroom . Sandra went to the garden . Daniel travelled to the kitchen . Mary journeyed to the kitchen . <eos> 
  - Question:  Where is Sandra ? <eos> 
   - Answer: kitchen / garden (ground-truth)

 it: 1760 | loss: 1.147 - 159.77s
 it: 1770 | loss: 1.170 - 160.79

 it: 2490 | loss: 1.148 - 233.95s
 it: 2500 | loss: 1.077 - 234.99s
 * test loss: 1.202 | acc: 0.450

 - Given text:  John journeyed to the kitchen . John travelled to the office . Daniel journeyed to the garden . Mary went to the bathroom . <eos> 
  - Question:  Where is John ? <eos> 
   - Answer: bathroom / office (ground-truth)

 it: 2510 | loss: 1.000 - 236.43s
 it: 2520 | loss: 1.070 - 237.34s
 it: 2530 | loss: 1.140 - 238.24s
 it: 2540 | loss: 1.065 - 239.13s
 it: 2550 | loss: 1.036 - 240.06s
 * test loss: 1.079 | acc: 0.510

 - Given text:  Sandra moved to the bedroom . Daniel went back to the bathroom . John moved to the garden . Daniel moved to the bedroom . John got the milk there . Daniel journeyed to the bathroom . Daniel went to the bedroom . John journeyed to the bedroom . <eos> 
  - Question:  Where is the milk ? <eos> 
   - Answer: bedroom / bedroom (ground-truth)

 it: 2560 | loss: 1.074 - 241.38s
 it: 2570 | loss: 1.118 - 242.33s
 it: 2580 | loss: 1.129 - 243.21s
 it:

 it: 3300 | loss: 1.133 - 328.71s
 * test loss: 1.076 | acc: 0.485

 - Given text:  Daniel took the milk there . Mary travelled to the bedroom . Daniel journeyed to the bedroom . Mary moved to the bathroom . John grabbed the football there . John went back to the bedroom . John left the football . Daniel discarded the milk . <eos> 
  - Question:  Where is the football ? <eos> 
   - Answer: bedroom / bedroom (ground-truth)

 it: 3310 | loss: 1.124 - 330.31s
 it: 3320 | loss: 1.023 - 331.39s
 it: 3330 | loss: 1.071 - 332.50s
 it: 3340 | loss: 1.155 - 333.66s
 it: 3350 | loss: 1.120 - 334.74s
 * test loss: 1.113 | acc: 0.455

 - Given text:  Mary journeyed to the bedroom . Mary went back to the kitchen . Daniel journeyed to the kitchen . John went back to the hallway . Mary travelled to the bedroom . Daniel went back to the garden . Mary went back to the hallway . John journeyed to the bathroom . <eos> 
  - Question:  Where is John ? <eos> 
   - Answer: bathroom / bathroom (ground-truth)


 it: 4030 | loss: 1.033 - 421.56s
 it: 4040 | loss: 1.018 - 422.63s
 it: 4050 | loss: 1.025 - 423.70s
 * test loss: 1.148 | acc: 0.475

 - Given text:  Mary journeyed to the bedroom . Mary went back to the office . Sandra went back to the hallway . Sandra went to the bedroom . John went back to the hallway . Daniel travelled to the bedroom . <eos> 
  - Question:  Where is Sandra ? <eos> 
   - Answer: bedroom / bedroom (ground-truth)

 it: 4060 | loss: 1.081 - 425.38s
 it: 4070 | loss: 1.001 - 426.51s
 it: 4080 | loss: 1.014 - 427.62s
 it: 4090 | loss: 1.084 - 428.83s
 it: 4100 | loss: 1.007 - 430.02s
 * test loss: 1.087 | acc: 0.460

 - Given text:  Sandra travelled to the office . Sandra went to the garden . Mary went back to the hallway . Mary journeyed to the kitchen . Sandra journeyed to the bedroom . John went to the bedroom . John moved to the garden . Daniel went back to the kitchen . Mary travelled to the bedroom . Daniel went back to the hallway . <eos> 
  - Question:  Where i

 it: 4810 | loss: 1.076 - 518.07s
 it: 4820 | loss: 1.059 - 519.26s
 it: 4830 | loss: 1.019 - 520.45s
 it: 4840 | loss: 1.060 - 521.60s
 it: 4850 | loss: 1.121 - 522.71s
 * test loss: 1.042 | acc: 0.530

 - Given text:  John moved to the bathroom . Sandra picked up the milk there . Daniel travelled to the garden . Daniel moved to the bedroom . Sandra picked up the apple there . Sandra went to the kitchen . Sandra discarded the apple . Sandra went to the garden . Sandra dropped the milk . Sandra went back to the kitchen . <eos> 
  - Question:  Where is the milk ? <eos> 
   - Answer: garden / garden (ground-truth)

 it: 4860 | loss: 1.028 - 524.31s
 it: 4870 | loss: 1.085 - 525.48s
 it: 4880 | loss: 1.037 - 526.59s
 it: 4890 | loss: 1.027 - 527.71s
 it: 4900 | loss: 1.003 - 528.85s
 * test loss: 1.087 | acc: 0.490

 - Given text:  Mary went back to the kitchen . John journeyed to the bedroom . Mary went back to the garden . Daniel went back to the bathroom . Daniel went to the bedroom . 

KeyboardInterrupt: 